# A-star

Based on this [blog](https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2)
and this [video](https://www.youtube.com/watch?v=-L-WgKMFuhE).

## Description

We use two lists of nodes: *closed* for nodes that have already been explored and *open* for nodes that remain to be explored.

1. Add the starting square (or node) to *open*.

1. Repeat the following:

    1. Look for the lowest F cost square on *open*. We refer to this as the current square.

    1. Switch it to *closed*.

    1. For each of the 8 squares adjacent to this current square:
        1. If it is not walkable or if it is on *closed*, ignore it. Otherwise do the following.
        1. If it isn’t on *open*, add it to *open*. Make the current square the parent of this square. Record the F, G, and H costs of the square.
        1. If it is on *open* already, check to see if this path to that square is better, using G cost as the measure. A lower G cost means that this is a better path. If so, change the parent of the square to the current square, and recalculate the G and F scores of the square. If you are keeping *open* sorted by F score, you may need to resort the list to account for the change.
    1. Stop when you:
        1. Add the target square to *closed*, in which case the path has been found, or Fail to find the target square, and *open* is empty. In this case, there is no path.
1. Save the path. Working backwards from the target square, go from each square to its parent square until you reach the starting square. That is your path.

## Pseudocode

![](Screenshot_A_star2.png)

## Maze

In [1]:
# 0 is air, 1 is wall
maze1 = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

start1 = (0, 0)
end1 = (7, 6)

## Nodes

In [2]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

## Algorithm

In [3]:
def astar(maze, start, end):
    """Returns a list of tuples as a path from start to end in maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node (with smallest f-value)
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current (with smallest f-value) off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # If the goal has been found, create the path
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path (from start to end)

        # Handle current by generating its children 
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range (if out of bounds continue)
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain (if a wall continue)
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1 # Also cost 1 for diagonals
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list (with a shorter path)
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)

## Find path in maze

In [4]:
path = astar(maze1, start1, end1)
print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]
